<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TrabajoN1Parte2Cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile Matrices.cpp
#include <iostream>
#include <cstdlib>
#include <thread>
#include <vector>
#include <random>
using namespace std;
#define MAX_DIMENSION 20
#define MIN_DIMENSION 5
#define MAX_VALOR 32
#define MIN_VALOR -32

void inicializarMatriz(int n, int ***matriz, int min, int max)
{
  (*matriz) = (int**)malloc(n*sizeof(int*));    
  random_device device;
  mt19937 generador(device());
  uniform_int_distribution<> distribucion(min, max);    
  for(int i=0; i<n; i++)
  {
    (*matriz)[i] = (int*)malloc(n*sizeof(int));
    for(int j=0; j<n; j++)          
      (*matriz)[i][j] = distribucion(generador);         
  }  
}

void calcularProducto(int n, int fila, int **matrizA, int **matrizB, int **matrizC)
{
  for(int i=0; i<n; i++)
    for(int j=0; j<n; j++)
      matrizC[fila][i] += matrizA[fila][j] * matrizB[j][i];       
}

void mostrarMatriz(int **matriz, int n, string nombre)
{
  cout<<endl<<"Matriz "<<nombre<<":"<<endl;
  for(int i=0; i<n; i++)
  {    
    cout<<"| ";
    for(int j=0; j<n; j++)
    {
      if(matriz[i][j] < -999)
        cout<<matriz[i][j]<<" ";
      else if (matriz[i][j] < -99 || matriz[i][j] > 999)
        cout<<" "<<matriz[i][j]<<" ";    
      else if (matriz[i][j] < -9 || matriz[i][j] > 99)
        cout<<"  "<<matriz[i][j]<<" "; 
      else if (matriz[i][j] < 0 || matriz[i][j] > 9)
        cout<<"   "<<matriz[i][j]<<" ";
      else if (matriz[i][j] < 10)
        cout<<"    "<<matriz[i][j]<<" ";                                  
    }       
    cout<<"  |"<<endl;
  }
}

int main(int argc, char *argv[])
{      
  //VERIFICO QUE SE HAYA ENVIADO EL NUMERO DE FILAS/COLUMNAS
  if( argc < 2 ) 
  {
    cout<<"Debe ingresar el numero de filas/columnas, dentro del rango de " <<MIN_DIMENSION<< " a " << MAX_DIMENSION <<endl;
    exit(1);
  }

  int n = atoi(argv[1]); //DIMENSIN DE LA MATRIZ    

  //VERIFICO QUE SE HAYA ENVIADO EL NUMERO DENTRO DE LOS LIMITES ESTABLECIDOS
  if( n < MIN_DIMENSION || n > MAX_DIMENSION ) 
  {
    cout<<"El numero de filas/columnas debe estar dentro del rango de " <<MIN_DIMENSION<< " a " << MAX_DIMENSION <<endl; 
    exit(1);
  }    

  //CREO LAS Matrices
  int **A, **B, **CS, **CH;   

  //INICIALIZO LAS MATRICES, A Y B CON VALORES RANDOM, CS Y CH EN 0
  inicializarMatriz(n, &A, MIN_VALOR, MAX_VALOR);
  inicializarMatriz(n, &B, MIN_VALOR, MAX_VALOR);
  inicializarMatriz(n, &CS, 0, 0);
  inicializarMatriz(n, &CH, 0, 0);
                
  //CALCULO VALORES DE CS (SECUENCIAL)
  for(int i=0; i<n; i++)
    calcularProducto(n, i, A, B, CS);    
    
  //CALCULO VALORES DE CH (CON CONCURRENCIA)
  //CREO UN VECTOR PARA ALMACENAR LAS REFERENCIAS A LOS HILOS CREADOS
  vector<thread> hilos;

  //CREO N HILOS
  for(int i=0; i<n; i++)     
    hilos.push_back(thread(calcularProducto, n, i, A, B, CH));                 
           
  //ESPERO A LOS N HILOS 
  for(int i=0; i<n; i++) 
    hilos[i].join();

  //MUESTRO LOS RESULTADOS
  mostrarMatriz(A,n,"A");    
  mostrarMatriz(B,n,"B");
  mostrarMatriz(CS,n,"CS"); 
  mostrarMatriz(CH,n,"CH"); 

  return 0;
}

Writing Matrices.cpp


In [2]:
!g++ -pthread -o Matrices Matrices.cpp

In [4]:
!./Matrices 10


Matriz A:
|   -16   -28    -6    12   -12   -30    10     3     2   -21   |
|    26   -16    26    -5   -14   -29    12   -21    -1   -16   |
|   -10    13    19     3    -2     7   -22   -25     7    21   |
|    26     9    31    -6    16   -26     7   -18    24    24   |
|   -16   -23     3   -32     5    -1    19    -4    17   -10   |
|    -5   -27    32    -6    10   -18   -26    11   -29   -23   |
|    32     7   -31   -13   -32    31     3   -10    -5   -14   |
|   -12    11   -32   -13    -6    -8    26     9    -7    24   |
|   -13     7   -21    -5    16    -2    -4   -15   -29    26   |
|   -17    17   -20     0    21     4     3     2   -27   -17   |

Matriz B:
|     1   -20   -21   -28    23    -1    27   -26   -10    -3   |
|     7     2    10    21   -16   -23    25   -24   -28     0   |
|   -23    22   -29    21    15    19   -20    -2    30    -6   |
|   -23     9    15    -1    14   -26   -14    31     3     8   |
|    31   -28   -23     9    -3   -13    -8    -1     